In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import logging
from tensorflow.keras.layers import *
import tensorflow as tf
import keras
from keras import Model
from keras import regularizers
from keras.layers import Dense, Dropout
from keras import layers
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
import pathlib
import ast
import json
import coral_ordinal as coral
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm
from imblearn.over_sampling import SMOTENC
from numpy import loadtxt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import torch
from torch.utils.data import TensorDataset, DataLoader
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import scatter, figure
import math
import numpy as np
import math
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.pyplot import scatter, figure
sns.set(rc={'figure.figsize':(9, 7)})
from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator


# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# Coral Ordinal Function and Custom metric defining here
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
MODEL_PATH = '../data/models/'
DATA_DIR = '../data/'


2023-11-28 15:37:50.549782: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 15:37:50.602957: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 15:37:51.373242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Important functions for test accuracy and confusion matrix
def predict_get_f1_accuracy(y_true, y_pred, num_classes):
    # Resetting the index
    y_true = y_true.reset_index(drop=True)

    # Function for custom distance accuracy metric
    def accuracy_metric(y_true, y_pred):
        a = []

        for i in range(len(y_true)):
            yt = y_true[i]
            yp = y_pred[i]

            temp = 1 - (np.abs(yt - yp) / num_classes)
            a.append(temp)
        # print(a)
        accuracy = sum(a) / len(y_true)

        return accuracy

    # Getting the f1 score using sklearn.metrics.precision_recall_fscore_support
    f1_score_test = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # Getting the custom distance accuracy metric
    custom_accuracy_test = accuracy_metric(y_true, y_pred)

    # Returing the values
    return f1_score_test[2], custom_accuracy_test


def save_confusion_matrix_func(y_test, yp, title, path, route_id):
    cm = confusion_matrix(y_test, yp)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    # Plot the confusion matrix
    fig, ax = plt.subplots(figsize=(8, 6))
    _ = disp.plot(ax=ax)
    # Add a title to the plot
    ax.set_title(title + " - Real Count")
    plt.savefig(f'{path}/CM_real_count_{route_id}.jpg')

    # Getting the percentage CM
    cm = ((cm * 100) / (cm.sum(axis=1)[:, np.newaxis]))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    # Plot the confusion matrix
    fig, ax = plt.subplots(figsize=(8, 6))
    _ = disp.plot(ax=ax)
    # Add a title to the plot
    ax.set_title(title + " - Percentages")
    plt.savefig(f'{path}/CM_percentage_{route_id}.jpg')



In [3]:

version = 'stl_1'
log_version = 'stl_1'
EPOCHS = 200

DATA_DIR = '../data'

pd.set_option('display.max_columns', None)

In [4]:
path = '../data/new_data/data1/'

train_df = pd.read_csv(f'{path}/Data1_Train.csv')
val_df = pd.read_csv(f'{path}/Data1_Val.csv')
test_df = pd.read_csv(f'{path}/Data1_Test.csv')

In [5]:
tensor_train_data = torch.Tensor(train_df.Input1).unsqueeze(1)
tensor_train_label = torch.Tensor(train_df.Output)

tensor_val_data = torch.Tensor(val_df.Input1).unsqueeze(1)
tensor_val_label = torch.Tensor(val_df.Output)

tensor_test_data = torch.Tensor(test_df.Input1).unsqueeze(1)
tensor_test_label = torch.Tensor(test_df.Output)

train_dataset = TensorDataset(tensor_train_data, tensor_train_label)
val_dataset = TensorDataset(tensor_val_data, tensor_val_label)


In [6]:
models = {}

In [7]:
import torch
import torch.optim as optim
from ray import tune
from ray.tune.schedulers import ASHAScheduler


class DeepEnsembleNet(nn.Module):
    def __init__(self, num_features, hidden_size, n_layers):
        super(DeepEnsembleNet, self).__init__()

        layers = []
        input_size = num_features

        # Create 'n' hidden layers
        for _ in range(n_layers):
            layers.append(nn.Linear(input_size, hidden_size))
            input_size = hidden_size  # The output of each layer is the input to the next

        self.layers = nn.ModuleList(layers)
        self.mu = nn.Linear(hidden_size, 1)
        self.var = nn.Linear(hidden_size, 1)

    def forward(self, x):
        for layer in self.layers:
            x = torch.relu(layer(x))
        mu = self.mu(x)
        var = torch.exp(self.var(x))
        return mu, var
    

In [8]:
def train_de(config):

    hidden_size = int(config['hidden_size'])
    num_layers = int(config['num_layers'])
    patience = int(config['patience'])
    batch_size = int(config['batch_size'])
    num_models = int(config['num_models'])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    model_name = f'hs_{hidden_size}_nl_{num_layers}_do_{config["num_models"]}_ep_{config["epochs"]}_p_{config["patience"]}_lr_{config["lr"]}_bs_{batch_size}.pt'

    models[f'{model_name}']= DeepEnsembleNet(config["num_features"], hidden_size, num_layers)

    device = 'cpu'
    models[f'{model_name}'].to(device)
    
    best_model_dir = f'./models/'
    pathlib.Path(best_model_dir).mkdir(parents=True, exist_ok=True) 
    best_model_path = f'{best_model_dir}/hs_{hidden_size}_nl_{num_layers}_do_{config["num_models"]}_ep_{config["epochs"]}_p_{config["patience"]}_lr_{config["lr"]}_bs_{batch_size}'
    best_val_loss = float('inf')
    epochs_no_improve = 0

    # Construct ensemble
    deep_ensemble = [DeepEnsembleNet(config["num_features"], hidden_size, num_layers).to(device) for i in range(num_models)]
    criterion = torch.nn.GaussianNLLLoss(eps=1e-02)
    optimizers = [optim.Adam(m.parameters(), lr=config["lr"]) for m in deep_ensemble]

    for epoch in range(config['epochs']):
        # Train loop
        for batch in train_loader:
            x = batch[0].to(device)
            y = batch[1].to(device)

            losses = []
            mus = []
            vars = []
            for i, model in enumerate(deep_ensemble):
                optimizers[i].zero_grad()
                mu, var = model(x)
                loss = criterion(mu, y, var)
                loss.backward()
                optimizers[i].step()

                losses.append(loss.item())
                mus.append(mu)
                vars.append(var)
            loss = sum(losses)/len(losses)

        # Validation loop to monitor early stopping
        model.eval()  # Set the model to evaluation mode
        val_losses = []
        for batch in val_loader:  # Assuming you have a validation data loader
            x = batch[0].to(device)
            y = batch[1].to(device)

            with torch.no_grad():  # No gradient calculation for validation data
                for model in deep_ensemble:
                    mu, var = model(x)
                    val_loss = criterion(mu, y, var)
                    val_losses.append(val_loss.item())
        avg_val_loss = sum(val_losses) / len(val_losses)

        # Early stopping logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f'Early stopping triggered at epoch {epoch}')
                break

        print(f'Epoch: {epoch}|Train Loss: {loss}|Val Loss: {avg_val_loss}')
    
    tune.report(train_loss=loss,best_val_loss=best_val_loss)


In [9]:


def main(num_samples=100, max_num_epochs=500):
    

    ray.init()

    config = {
        "num_features": 1,
        "hidden_size": tune.quniform(1, 200, 1),
        "num_layers": tune.quniform(2, 10, 1),
        "num_models": 5,
        "lr": tune.loguniform(1e-4, 1e-1),
        "epochs": max_num_epochs,
        "patience": tune.quniform(1, 50, 1),
        "batch_size": tune.quniform(1, 5, 1)
    }

    from ray.tune.search import ConcurrencyLimiter
    from ray.tune.search.bayesopt import BayesOptSearch
    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
    algo = ConcurrencyLimiter(algo, max_concurrent=40)

    result = tune.run(
            tune.with_parameters(train_de),
            config=config,
            resources_per_trial={"cpu": 1},
            metric="best_val_loss",
            mode="min",
            search_alg=algo,
            num_samples=num_samples,
            # max_concurrent_trials=20,
            local_dir= f'/home/sgupta/WORK/Triplevel_transformer_model/baselines/hyperparameter_search',
            name=f"experiment_de",
            max_failures=7,
            raise_on_failed_trial=False
        )

    temp = result.dataframe()   
    hyperparams_path = f'./hyperparameter_search/'
    temp.to_csv(f"{hyperparams_path}/RAY_RESULTS_de.csv")


In [10]:
import ray

main(200,500)
#  Stop Ray
ray.shutdown()

2023-11-28 15:37:57,286	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-11-28 15:37:58,379	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-11-28 15:37:58,397	WARNING bayesopt_search.py:421 -- BayesOpt search does not support quantization. Dropped quantization.
2023-11-28 15:37:58,398	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2023-11-28 15:37:58,398	WARNING bayesopt_search.py:421 -- BayesOpt search does not support quantization. Dropped quantization.
2023-11-28 15:37:58,399	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2023-11-28 15:37:58,400	WARNING bayesopt_search.py:421 -- BayesOpt search does not supp

(train_de pid=3622506) Epoch: 0|Train Loss: nan|Val Loss: nan
(train_de pid=3622506) Epoch: 1|Train Loss: nan|Val Loss: nan
(train_de pid=3622506) Epoch: 2|Train Loss: nan|Val Loss: nan
(train_de pid=3622506) Epoch: 3|Train Loss: nan|Val Loss: nan
(train_de pid=3622506) Epoch: 4|Train Loss: nan|Val Loss: nan
(train_de pid=3622506) Epoch: 5|Train Loss: nan|Val Loss: nan


Trial name,best_val_loss,train_loss
train_de_017ef564,0.156542,0.220031
train_de_018155f0,0.173418,0.0616801
train_de_01a68e07,0.189641,0.0322618
train_de_020ee1db,inf,nan
train_de_02d41194,0.191095,0.290188
train_de_03721124,0.191323,0.580934
train_de_03cbb2cd,inf,nan
train_de_04648e91,0.183294,0.0592827
train_de_055387b0,0.189057,0.114952
train_de_0596463a,0.275259,0.456442


NaN or Inf found in input tensor.


(train_de pid=3622506) Early stopping triggered at epoch 7
(train_de pid=3622506) Epoch: 6|Train Loss: nan|Val Loss: nan [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(train_de pid=3622506) Epoch: 0|Train Loss: -0.006479149684309959|Val Loss: 0.20723233990944348
(train_de pid=3622506) Epoch: 1|Train Loss: 0.7983434557914734|Val Loss: 0.2010678650668034
(train_de pid=3622506) Epoch: 2|Train Loss: 0.033543243631720544|Val Loss: 0.18985645793951475
(train_de pid=3622506) Epoch: 3|Train Loss: -0.018592395633459092|Val Loss: 0.2082338288426399
(train_de pid=3622506) Epoch: 4|Train Loss: -0.1430385708808899|Val Loss: 0.19200145974755287
(train_de pid=3622506) Epoch: 5|Train Loss: -0.0017942704260349273|Val Loss: 0.1951419609097334
(train_de pid=3622506) Epoch: 6|Train Loss: 0.3987118065357208|Val Loss: 0.203508022312

...

NaN or Inf found in input tensor.


(train_de pid=3629615) Epoch: 11|Train Loss: nan|Val Loss: nan
(train_de pid=3631729) Epoch: 110|Train Loss: 0.17328405876954397|Val Loss: 0.20265050679810492 [repeated 19x across cluster]
(train_de pid=3630145) Early stopping triggered at epoch 32 [repeated 2x across cluster]
(train_de pid=3629615) Epoch: 12|Train Loss: nan|Val Loss: nan
(train_de pid=3626657) Epoch: 27|Train Loss: 0.15140687301754951|Val Loss: 0.20992065948324606 [repeated 16x across cluster]
(train_de pid=3627776) Early stopping triggered at epoch 54
(train_de pid=3633324) Early stopping triggered at epoch 49
(train_de pid=3628737) Epoch: 43|Train Loss: 0.02521194357957159|Val Loss: 0.20359267347625323 [repeated 15x across cluster]
(train_de pid=3629615) Epoch: 13|Train Loss: nan|Val Loss: nan
(train_de pid=3631729) Epoch: 127|Train Loss: 0.30850311120351154|Val Loss: 0.18837994990833834 [repeated 16x across cluster]
(train_de pid=3629615) Early stopping triggered at epoch 14


NaN or Inf found in input tensor.


(train_de pid=3632074) Epoch: 30|Train Loss: 0.5254493951797485|Val Loss: 0.21279542344725794 [repeated 19x across cluster]
(train_de pid=3631729) Early stopping triggered at epoch 137
(train_de pid=3626920) Epoch: 43|Train Loss: -2.269048035144806|Val Loss: -1.8939063671231269 [repeated 14x across cluster]
(train_de pid=3626920) Epoch: 45|Train Loss: -2.2662203311920166|Val Loss: -1.9955154870450498 [repeated 11x across cluster]
(train_de pid=3628561) Epoch: 35|Train Loss: 0.20439490489661694|Val Loss: 0.15994181238813326 [repeated 14x across cluster]
(train_de pid=3626519) Epoch: 12|Train Loss: -0.3008905202150345|Val Loss: 0.1950423282571137 [repeated 13x across cluster]
(train_de pid=3631571) Epoch: 36|Train Loss: -2.2932206903185164|Val Loss: -1.8497094402994427 [repeated 11x across cluster]
(train_de pid=3628561) Epoch: 39|Train Loss: -0.3611926957964897|Val Loss: 0.1815343400835991 [repeated 13x across cluster]
(train_de pid=3626920) Epoch: 57|Train Loss: -2.2980878353118896|Val

2023-11-28 16:35:13,092	INFO tune.py:1148 -- Total run time: 3434.71 seconds (3434.48 seconds for the tuning loop).
